In [1]:
#import statements
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime as dt
import math
import json

here = os.getcwd()

In [22]:
#loads species and map key data sets
#source: me
afilename = here+"/data/2020/2020datasimp.csv"
samples = pd.read_csv(afilename)

afilename = here+"/data/2020/map-keys-2020.csv"
map_keys = pd.read_csv(afilename)

#loads the latest (2014) list of black list/watch list of invasive species for Switzerland from Flora Helvetica,
#source: https://www.infoflora.ch/fr/neophytes/listes-et-fiches.html
afilename = here+"/data/florahelvetica/BL_WL_2014_modified.csv"
invasives = pd.read_csv(afilename)

#loads the latest list of undersampled species for Switzerland
#source: https://www.infoflora.ch/fr/participer/que-signaler.html
afilename = here+"/data/florahelvetica/taxa_sous_echantillonnes.csv"
tse = pd.read_csv(afilename)

#loads the iucn redlist 2019 classification for switzerland
#source: https://www.infoflora.ch/fr/flore/conservation-des-especes/liste-rouge.html
afilename = here+"/data/florahelvetica/CH-RLreg_Tracheophyta_2019.csv"
redlist = pd.read_csv(afilename)

#loads a downloaded data set from GBIF
afilename = here+"/data/florahelvetica/gbif_species_list.csv"
gbif = pd.read_csv(afilename)

#loads the Welten-Sutter lists of species that cover Biel
#source: https://www.infoflora.ch/de/daten/artenliste-welten-sutter.html
myencoding = "utf-16"
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_151_Biel.csv"
ws151 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_253_Bielersee.csv"
ws253 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_300_Aarberg.csv"
ws300 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_301_Bueren.csv"
ws301 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_154_Grenchen.csv"
ws154 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_572_Beatenberg.csv"
ws572 = pd.read_csv(afilename, sep = ";", encoding = "utf-16")
afilename = here+"/data/florahelvetica/atlasws/AtlasWS_573_Interlaken.csv"
ws573 = pd.read_csv(afilename, sep = ";", encoding = "utf-16")

#loads the 4 5x5 km datasets for species that cover biel 
#source: https://www.infoflora.ch/de/daten/artenliste-5x5-km.html
afilename = here+"/data/florahelvetica/atlas5x5/Atlas5x5_585_215.csv"
a585215 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlas5x5/Atlas5x5_585_220.csv"
a585220 = pd.read_csv(afilename, encoding = myencoding)
afilename = here+"/data/florahelvetica/atlas5x5/Atlas5x5_580_220.csv"
a580220 = pd.read_csv(afilename, sep = ";", encoding = myencoding)
afilename = here+"/data/florahelvetica/atlas5x5/Atlas5x5_580_215.csv"
a580215 = pd.read_csv(afilename, sep = ";", encoding = myencoding)

In [3]:
#harmonizes the data format of species using the scientific binomial as slug (i.e. genus-species)
#note that this removes the subspecies from consideration. This is done as the surveys are conducted
#at the agg. level. Species slugs all stored in column "species1"

#format the 3rd party data sets.
def change_species(x):
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = F"wrong {x}"
    return data

#welten-sutter and 5X5 km lists
ws151["species1"] = ws151.species.map(lambda x: change_species(x))
ws300["species1"] = ws300.art.map(lambda x: change_species(x))
ws253["species1"] = ws253.species.map(lambda x: change_species(x))
ws301["species1"] = ws301.art.map(lambda x: change_species(x))
ws154["species1"] = ws154.species.map(lambda x: change_species(x))
ws572["species1"] = ws572.species.map(lambda x: change_species(x))
ws573["species1"] = ws573.species.map(lambda x: change_species(x))
a585215["species1"] = a585215.species.map(lambda x: change_species(x))
a585220["species1"] = a585220.species.map(lambda x: change_species(x))
a580215["species1"] = a580215.species.map(lambda x: change_species(x))
a580220["species1"] = a580220.species.map(lambda x: change_species(x))

#invasive, under-sampled, gbif, and red list species lists
invasives.rename(columns={"Latin":"name"}, inplace = True)
invasives["species"] = invasives.name.map(lambda x: change_species(x))
tse["species"] = tse.taxon.map(lambda x: change_species(x))
redlist["species1"] = redlist["species"].astype(str)
redlist["species1"] = redlist.scientific_name.map(lambda x: x.replace(" ", "-"))
redlist["species1"] = redlist.species1.map(lambda x: x.lower())
gbif["species1"] = gbif.species.map(lambda x: change_species(x))

In [4]:
#makes a master list of the overlapping Welten-Sutter areas and Infoflora 5X5 kilomter areas where the large majority of surveys were conducted.
#can add locations uniquely as opportunistic surveys in locations other than the region immediately around Biel are conducted.

#create an identifiying column name in each data set.
ws = "ws"
ws151[ws] = "ws151"
ws253[ws] = "ws253"
ws300[ws] = "ws300"
ws301[ws] = "ws301"
ws154[ws] = "ws154"
ws572[ws] = "ws572"
ws573[ws] = "ws573"
a585215[ws] = "a585215"
a585220[ws] = "a585220"
a580215[ws] = "a580215"
a580220[ws] = "a580220"

#make a tuple out of the file name column the species slug column for each data set 
at = "atuple"
ws151[at] = list(zip(ws151.species1,ws151.ws))
ws253[at] = list(zip(ws253.species1,ws253.ws))
ws300[at] = list(zip(ws300.species1,ws300.ws))
ws301[at] = list(zip(ws301.species1,ws301.ws))
ws154[at] = list(zip(ws154.species1,ws154.ws))
ws572[at] = list(zip(ws572.species1,ws572.ws))
ws573[at] = list(zip(ws573.species1,ws573.ws))
a585215[at] = list(zip(a585215.species1,a585215.ws))
a585220[at] = list(zip(a585220.species1,a585220.ws))
a580215[at] = list(zip(a580215.species1,a580215.ws))
a580220[at] = list(zip(a580220.species1,a580220.ws))

#set ws151 as the initial list
#species records are not unique in the imported data sets, thus must select the unique species, location tuples.
mymasterlist = ws151.atuple.unique()
a = set(mymasterlist)
b = set(ws253.atuple.unique())
c = set(ws300.atuple.unique())
d = set(ws301.atuple.unique())
e = set(ws154.atuple.unique())
f = set(ws572.atuple.unique())
g = set(ws573.atuple.unique())
h = set(a585215.atuple.unique())
i = set(a585220.atuple.unique())
j = set(a580215.atuple.unique())
k = set(a580220.atuple.unique())

#creates a dictionary with species slug as key and list of places identified as the corresponding value.
def unique_species(alist):
    data = {x[0]:[] for x in alist}
    for a in alist:
        data[a[0]].append(a[1])
    return data
newmasterlist = unique_species([*a,*b, *c, *d, *e, *f, *g, *h, *i, *j, *k])

#exports the dictionary to a .json file
here = os.getcwd()
here1 = F"{here}/output/ws_list.json"
with open(F"{here1}","w") as afile:
    json.dump(newmasterlist,afile)

In [13]:
#makes a dictionary of the switzerland wide species list of species that are indigenous (redlist), undersampled (tse), invasive (invasives)
#same method as previous
tse["tse"] = "tse"
invasives["inv"] = "inv"
redlist["rl"] = "rl"
invasives[at] = list(zip(invasives.species, invasives.list_2014))
redlist[at] = list(zip(redlist.species1, redlist.rl))
tse[at] = list(zip(tse.species,tse.tse))
a = set(redlist.atuple.unique())
b = set(invasives.atuple.unique())
c = set(tse.atuple.unique())
def species_key(alist):
    data = {x[0]:[] for x in alist}
    for a in alist:
        data[a[0]].append(a[1])
    return data
newmasterlist = unique_species([*a,*b])

#exports the data to a .json file for use
here = os.getcwd()
here1 = F"{here}/output/flora_list.json"
with open(F"{here1}","w") as afile:
    json.dump(newmasterlist,afile)

#using the gbif dataset appends higher order taxonomic information to the species slug
#modify previous code to output all elements of a tuple to a .json file (not just element [1])
gbif["gbif"] = "gbif"    
gbif[at] = list(zip(gbif.species1, gbif.gbif, gbif.genus, gbif.family, gbif.order, gbif.theclass))
a = set(gbif.atuple.unique())

In [14]:
#correct wrong species name inputs in the samples data
#samples list
replacedict = {
    'verbanum bonariensis ':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "senecio jacobaea": "jacobaea vulgaris",
    "oenothera biennis ": "oenothera biennis",
    "oenothera biennis agg.": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha-nepeta":"clinopodium nepeta",
    "polygonum-persicaria":"persicaria maculosa",
    "sorbus-aria":"aria edulis",
    "taraxacum": "taraxacum officinale"
}
def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data
samples["species2"] = samples.sci.map(lambda x: new_func(x, replacedict))
samples["species1"] = samples.species2.map(lambda x: change_species(x))

In [15]:
#check sample data against data in gbif and redlist datasets
a = samples.species1.unique()
b = gbif.species1.unique()
c = redlist.species1.unique()
d = invasives.species.unique()
samples_not_rl = [x for x in samples.species1.unique() if x not in redlist.species1.unique()]
samples_not_gbif = [x for x in samples.species1.unique() if x not in gbif.species1.unique()]
samples_neither = [x for x in samples.species1.unique() if (x not in b and x not in c and x not in d)]

In [16]:
#correct incorrectly formatted date inputs in the sample dataset
def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data
samples['new_date'] = samples.date.map(lambda x: change_string(x))


def make_timestamp(x):
    try:        
        data = dt.datetime.strptime(x, "%Y-%m-%d")        
    except:        
        data = 'no date'
    
    return data
samples['check_date'] = samples.new_date.map(lambda x: make_timestamp(x))
samples['date'] = samples.new_date

print(F"\nThese are the records in a with no valid date:\n\n{samples[samples.check_date == 'no date']}\n")


These are the records in a with no valid date:

Empty DataFrame
Columns: [place, date, time, sci, name, species2, species1, new_date, check_date, place1, waterbody, wsnum]
Index: []



In [24]:
#sample place columns

#makes place slug in new column "place1" for the locations with more than one word.
def change_place(x):
    data = x.split(" ")
    data = "-".join(data)
    return data
samples["place1"] = samples.place.map(lambda x: change_place(x))

#adds identifying closest waterbody to the list of place names
#note that suze_b = suze in biel, suz_u = suze upstream from biel.
def add_waterbody(x):
    pattern1 = "sund"
    pattern2 = "frinv"
    pattern3 = "orvine"
    pattern4 = "cheyre"
    pattern5 = "ligerz"
    pattern6 = "twann"
    pattern7 = "tauben"
    pattern8 = "lucherz"
    pattern9 = "rondc"
    pattern10 = "weisse"
    if pattern1 in x or pattern10 in x:
        data = "thunersee"
    elif pattern2 in x or pattern7 in x or pattern9 in x:
        data = "suze_u"
    elif pattern3 in x:
        data = "orvine"
    elif pattern4 in x:
        data = "neuenburgersee"
    elif pattern5 in x or pattern6 in x or pattern8 in x:
        data = "bielersee"
    else:
        data = "suze_b"
    return data
samples["waterbody"] = samples.place.map(lambda x: add_waterbody(x))

#add welten-sutter number to sample location
def addwsnum(x):
    pattern1 = "sund"
    pattern2 = "cheyre"
    pattern3 = "lucherz"
    pattern4 = "weisse"
    pattern5 = "falls"
    if pattern1 in x:
        data = "ws572"
    elif pattern2 in x:
        data = "ws226"
    elif pattern3 in x:
        data = "ws252"
    elif pattern4 in x:
        data = "ws573"
    elif pattern5 in x:
        data = "not-relevant"
    else:
        data = "ws151"
    return data

samples["wsnum"] = samples.place.map(lambda x: addwsnum(x))
samples.tail()

#using place as key, append the longitude, lattitude and waterbody to each sample. (replacing the existing waterbody column)
map_keys.head()

,lon,lat,place,waterbody
0,7.253285,47.138823,jura-2,suze
1,7.791570,46.685610,sundgraben-port,thunersee
2,7.252867,47.137585,salome-3,suze
3,7.252760,47.137273,salome-4,suze
4,7.252618,47.136619,alleestrasse-4,suze


In [19]:
samples.head()

,place,date,time,sci,name,species2,species1,new_date,check_date,place1,waterbody,wsnum
0,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé,plantago lanceolata,plantago-lanceolata,2020-09-02,2020-09-02,alleestrasse-1,suze_b,ws151
1,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire,centaurea nigra,centaurea-nigra,2020-09-02,2020-09-02,alleestrasse-1,suze_b,ws151
2,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen,plantago media,plantago-media,2020-09-02,2020-09-02,alleestrasse-1,suze_b,ws151
3,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc,chenopodium album agg.,chenopodium-album,2020-09-02,2020-09-02,alleestrasse-1,suze_b,ws151
4,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée,centaurea jacea agg.,centaurea-jacea,2020-09-02,2020-09-02,alleestrasse-1,suze_b,ws151
